In [1]:
import torch
from torch.utils import data
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

/home/dumerenkov/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/dumerenkov/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
ASSEMBLY_d = {}
chroms_d = {}
all_features_d = {}
groups_d = {}
feature_names_d = {}
ZDNA_d = {}
black_list_d = {}
DNA_d = {}
DNA_features_d = {}

# MM9

In [3]:
ASSEMBLY = "curax_14h_UNI_mm9"
chroms = [f'chr{i}' for i in list(range(1, 20)) + ['X', 'Y']]
all_features = sorted([i[:-4] for i in os.listdir('../data/mm9_features/sparse/') if i.endswith('.pkl')])
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in all_features if (i.split('_')[0] in groups)]
ZDNA = load(f'../data/mm9_zdna/sparse/{ASSEMBLY}.pkl')
black_list = load(f'../data/mm9_zdna/sparse/blacklist_mm9.pkl')


In [4]:
DNA = {chrom:load(f'../data/mm9_dna/sparse/{chrom}.pkl') for chrom in tqdm(chroms)}

DNA_features = {feture: load(f'../data/mm9_features/sparse/{feture}.pkl')
                for feture in tqdm(feature_names)}

for feature in tqdm(DNA_features):
    if set(DNA_features[feature].keys()) != set(chroms):
        for chrom in chroms:
            if chrom not in DNA_features[feature]:
                DNA_features[feature][chrom] = SparseVector(len(DNA[chrom]))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 873/873 [00:00<00:00, 195401.43it/s]


In [5]:
mode = 'mm9'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
all_features_d[mode] = all_features
groups_d[mode] = groups
feature_names_d[mode] = feature_names
ZDNA_d[mode] = ZDNA
black_list_d[mode] = black_list
DNA_d[mode] = DNA
DNA_features_d[mode] = DNA_features

# HG19

In [6]:
ASSEMBLY = "ZDNA_2016"
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]
all_features = sorted([i[:-4] for i in os.listdir('../data/hg19_features/sparse/') if i.endswith('.pkl')])
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in all_features if (i.split('_')[0] in groups)]
ZDNA = load(f'../data/hg19_zdna/sparse/{ASSEMBLY}.pkl')
black_list = load(f'../data/hg19_zdna/sparse/blacklist_hg19.pkl')


In [7]:
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

DNA_features = {feture: load(f'../data/hg19_features/sparse/{feture}.pkl')
                for feture in tqdm(feature_names)}

for feature in tqdm(DNA_features):
    if set(DNA_features[feature].keys()) != set(chroms):
        for chrom in chroms:
            if chrom not in DNA_features[feature]:
                DNA_features[feature][chrom] = SparseVector(len(DNA[chrom]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1054/1054 [00:00<00:00, 139088.74it/s]


In [8]:
mode = 'hg19'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
all_features_d[mode] = all_features
groups_d[mode] = groups
feature_names_d[mode] = feature_names
ZDNA_d[mode] = ZDNA
black_list_d[mode] = black_list
DNA_d[mode] = DNA
DNA_features_d[mode] = DNA_features

In [9]:
intersect = {i.upper() for i in DNA_features_d['mm9']} & {i.upper() for i in DNA_features_d['hg19']}

In [10]:
def to_bed(source, name):
    buf = []
    for chrm in source[name]:
        beds = source[name][chrm].indices[source[name][chrm].data != 0]
        data = source[name][chrm].data[source[name][chrm].data != 0].astype(float).astype(int)
        ends = np.append(source[name][chrm].indices[1:], 
              [source[name][chrm].shape])[source[name][chrm].data != 0]

        buf.extend([[chrm, beds[i], ends[i], data[i]] for i in range(len(beds))])
    buf = np.array(buf)
    df = pd.DataFrame([buf[:, 0], buf[:, 1], buf[:, 2], '', buf[:, 3], "", buf[:, 1], buf[:, 2], ""]).T
    df[3] = ''
    df[5] = "+"
    df[8] = ''
    return df

In [11]:
gen = 'hg19'

for name in tqdm([i for i in DNA_features_d[gen] if i.upper() in intersect and 'TFs and others' in i]):
    df = to_bed(DNA_features_d[gen], name)
    df.to_csv(f'tmp/{gen}_{name}.bed', sep = '\t', index=False, header=None)
#     break

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 470/470 [31:31<00:00,  4.02s/it]


In [11]:
def func(DNA_features_d, name, gen):
    df = to_bed(DNA_features_d, name)
    df.to_csv(f'tmp/{gen}_{name}.bed', sep = '\t', index=False, header=None)

In [13]:
from joblib import Parallel, delayed
gen = 'hg19'
Parallel(n_jobs = -1)(delayed(func)(DNA_features_d[gen], name, gen)
                     for name in tqdm([i for i in DNA_features_d[gen] 
                                       if i.upper() in intersect and 'TFs and others' in i]))



 41%|█████████████████████████████████████████████▊                                                                  | 192/470 [09:18<17:38,  3.81s/it]/home/dumerenkov/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/dumerenkov/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
 51%|█████████████████████████████████████████████████████████▏                                                      | 240/470 [13:53<17:14,  4.50s/it]/home/dumerenkov/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/dumerenkov/

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Data part

In [12]:
mode = 'hg19'
ASSEMBLY = ASSEMBLY_d[mode]
chroms = chroms_d[mode]
all_features = all_features_d[mode]
groups = groups_d[mode]
feature_names = feature_names_d[mode]
ZDNA = ZDNA_d[mode]
black_list = black_list_d[mode]
DNA = DNA_d[mode]
DNA_features = DNA_features_d[mode]

In [52]:
width = 512

np.random.seed(10)

ints_in = []
ints_out = []


for chrm in chroms:
    for st in trange(0, ZDNA[chrm].shape - width, width):
        interval = [st, min(st + width, ZDNA[chrm].shape)]
        N_count = sum([bp == "N" for bp in DNA[chrm][interval[0]:interval[1]]])
        bl_count = black_list[chrm][interval[0]:interval[1]].sum()
        if N_count > width / 2 or bl_count > 0:
            continue
        else:
            if ZDNA[chrm][interval[0]: interval[1]].any():
                ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
            else:
                ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])


                
                
print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 115963/115963 [00:04<00:00, 25221.92it/s]

629
5472235


In [53]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)), 
                                                    size=len(ints_in) * 20, replace=False)]
# ints_out = ints_out_full

print(len(ints_in))
print(len(ints_out))
#484 for len 1000 
#9680 for len 1000

#629 for len 512
#12580 for len 5121

629
12580


In [54]:
equalized = ints_in + ints_out

In [55]:
divisions = list(StratifiedKFold(5, shuffle=True, 
                                 random_state=42).split(equalized, [f"{elem[3]}_{elem[0]}"
                                         for i, elem 
                                         in enumerate(equalized)]))

In [19]:
dump([equalized, divisions], 'hg_divisions.pkl', 3)

['hg_divisions.pkl']

In [13]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [14]:
class Dataset(data.Dataset):
    def __init__(self, chroms, features, 
                 dna_source, features_source, 
                 labels_source, intervals, tokenizer):

        self.chroms = chroms
        self.features = features
        self.dna_source = dna_source
        self.features_source = features_source
        self.labels_source = labels_source
        self.intervals = intervals
        self.le = LabelBinarizer().fit(np.array([["A"], ["C"], ["T"], ["G"]]))
        self.configs = {
                        'ZHUNT_AS': {
                                'CG': 0, 'GC': 1, 'CA': 0, 'AC': 1, 
                                'TG': 0, 'GT': 1, 'TA': 1, 'AT': 1, 
                                'CC': 0, 'GG': 0, 'CT': 1, 'TC': 1, 
                                'GA': 1, 'AG': 1, 'AA': 1, 'TT': 1},
                       }
        seqs = (["A", "C", "T", "G"] + 
                ['AC', 'AT', 'AG', 'CT', 'CG', 'GT'] +
                ['AAC', 'ACC', 'AAT', 'ATT', 'AAG', 'AGG', 
                 'CCA', 'CAA', 'CCT', 'CTT', 'CCG', 'CGG', 
                 'TTA', 'TAA', 'TTC', 'TCC', 'TTG', 'TGG', 
                 'GGA', 'GAA', 'GGC', 'GCC', 'GGT', 'GTT'] +
                ['AAAC', 'AAAT', 'AAAG', 'CCCA', 'CCCT', 'CCCG',
                 'TTTA', 'TTTC', 'TTTG', 'GGGA', 'GGGC', 'GGGT'])
        self.tars = np.array([self.le.transform(list(i * 11)[:11]) for i in seqs])[:, ::-1, ::-1]
        # purine-pyrimidine
        self.tars = np.concatenate((self.tars, np.array([self.tars[4] + self.tars[9]])))
        self.tokenizer = tokenizer
        
        
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        ll = list(self.dna_source[chrom][begin:end].upper())
        y = self.labels_source[interval[0]][interval[1]: interval[2]]        
        
        
#         DNA PART
        
        dna_OHE = self.le.transform(ll)[None]
        
        res = pd.DataFrame(convolve(dna_OHE, self.tars)[:, 5:-5, 3].T / 11)
        res = (res.rolling(5, min_periods=1).max().values == 1).astype(int)
        
        
#         ZHUNT PART
        zhunts = []
        for key in self.configs:
            vec = np.array(ll)
            vec = np.vectorize(lambda x:self.configs[key].get(x, 0))(
                                    np.char.add(vec[1:], vec[:-1]))
            zhunts.append(np.concatenate([vec, [0]]))
        
        
        # FEATURES PART
        feature_matr = []
        for feature in self.features:
            source = self.features_source[feature]
            feature_matr.append(source[chrom][begin:end])
        
        # UNION
        if len(feature_matr) > 0:
            X = np.hstack((
                           res,
                           np.array(zhunts).T, 
                           np.array(feature_matr).T/1000)).astype(np.float32)
#             X = (np.array(feature_matr).T/1000).astype(np.float32)
        else:
            X = dna_OHE.astype(np.float32)
        
        #K-mer part
        
        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.Tensor(X), torch.Tensor(y).long(), ll, torch.LongTensor(encoded_k_mers), (chrom, begin, end)


In [15]:
import torch
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections
from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

In [16]:
VOCAB_FILES_NAMES = {"vocab_file": "vocab.txt"}




PRETRAINED_VOCAB_FILES_MAP = {"vocab_file": {"dna3": "https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-3/vocab.txt",
                                             "dna4": "https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-4/vocab.txt",
                                             "dna5": "https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-5/vocab.txt",
                                             "dna6": "https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/vocab.txt"}}



PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
                                          "dna3": 512,
                                          "dna4": 512,
                                          "dna5": 512,
                                          "dna6": 512}

PRETRAINED_INIT_CONFIGURATION = {
    "dna3": {"do_lower_case": False},
    "dna4": {"do_lower_case": False},
    "dna5": {"do_lower_case": False},
    "dna6": {"do_lower_case": False}}

VOCAB_KMER = {
    "69": "3",
    "261": "4",
    "1029": "5",
    "4101": "6",}


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

class DNATokenizer(PreTrainedTokenizer):
    r"""
    Constructs a BertTokenizer.
    :class:`~transformers.BertTokenizer` runs end-to-end tokenization: punctuation splitting + wordpiece
    Args:
        vocab_file: Path to a one-wordpiece-per-line vocabulary file
        do_lower_case: Whether to lower case the input. Only has an effect when do_basic_tokenize=True
        do_basic_tokenize: Whether to do basic tokenization before wordpiece.
        max_len: An artificial maximum length to truncate tokenized sequences to; Effective maximum length is always the
            minimum of this value (if specified) and the underlying BERT model's sequence length.
        never_split: List of tokens which will never be split during tokenization. Only has an effect when
            do_basic_tokenize=True
    """

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES


    def __init__(
        self,
        vocab_file,
        do_lower_case=False,
        never_split=None,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        tokenize_chinese_chars=True,
        max_len = 512,
        **kwargs
    ):
        """Constructs a BertTokenizer.
        Args:
            **vocab_file**: Path to a one-wordpiece-per-line vocabulary file
            **do_lower_case**: (`optional`) boolean (default True)
                Whether to lower case the input
                Only has an effect when do_basic_tokenize=True
            **do_basic_tokenize**: (`optional`) boolean (default True)
                Whether to do basic tokenization before wordpiece.
            **never_split**: (`optional`) list of string
                List of tokens which will never be split during tokenization.
                Only has an effect when do_basic_tokenize=True
            **tokenize_chinese_chars**: (`optional`) boolean (default True)
                Whether to tokenize Chinese characters.
                This should likely be deactivated for Japanese:
                see: https://github.com/huggingface/pytorch-pretrained-BERT/issues/328
        """
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )
        self.vocab = load_vocab(vocab_file)
        self.max_len = max_len
        #self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        #self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'. To load the vocabulary from a Google pretrained "
                "model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`".format(vocab_file)
            )
        
        self.kmer = VOCAB_KMER[str(len(self.vocab))]
        self.ids_to_tokens = collections.OrderedDict([(ids, tok) for tok, ids in self.vocab.items()])
        self.basic_tokenizer = BasicTokenizer(
                do_lower_case=do_lower_case, never_split=never_split, tokenize_chinese_chars=tokenize_chinese_chars
            )

    @property
    def vocab_size(self):
        return len(self.vocab)

    def _tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text, never_split=self.all_special_tokens):
                split_tokens.append(token)
        # print(split_tokens)
        return split_tokens

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        return self.vocab.get(token, self.vocab.get(self.unk_token))

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        return self.ids_to_tokens.get(index, self.unk_token)

    def convert_tokens_to_string(self, tokens):
        """ Converts a sequence of tokens (string) in a single string. """
        out_string = " ".join(tokens).replace(" ##", "").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A BERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]

        if token_ids_1 is None:
            if len(token_ids_0) < 510:
                return cls + token_ids_0 + sep
            else:
                output = []
                num_pieces = int(len(token_ids_0)//510) + 1
                for i in range(num_pieces):
                    output.extend(cls + token_ids_0[510*i:min(len(token_ids_0), 510*(i+1))] + sep)
                return output

        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 1 for a special token, 0 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        
        if len(token_ids_0) < 510:
            return [1] + ([0] * len(token_ids_0)) + [1]
        else:
            output = []
            num_pieces = int(len(token_ids_0)//510) + 1
            for i in range(num_pieces):
                output.extend([1] + ([0] * (min(len(token_ids_0), 510*(i+1))-510*i)) + [1])
            return output
            return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            if len(token_ids_0) < 510:
                return len(cls + token_ids_0 + sep) * [0]
            else:
                num_pieces = int(len(token_ids_0)//510) + 1
                return (len(cls + token_ids_0 + sep) + 2*(num_pieces-1)) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, vocab_path):
        """Save the tokenizer vocabulary to a directory or file."""
        index = 0
        if os.path.isdir(vocab_path):
            vocab_file = os.path.join(vocab_path, VOCAB_FILES_NAMES["vocab_file"])
        else:
            vocab_file = vocab_path
        with open(vocab_file, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.vocab.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(vocab_file)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
        return (vocab_file,)

In [63]:
for MODEL_NUMBER in range(5):

    train_inds, test_inds = divisions[MODEL_NUMBER]
    train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]

    random.shuffle(train_intervals)
    random.shuffle(test_intervals)
    
    train_dataset = Dataset(chroms, 
                        [i for i in feature_names if i.upper() in intersect], 
                       DNA, DNA_features, ZDNA, train_intervals, 
                        DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))

    test_dataset = Dataset(chroms, 
                       [i for i in feature_names if i.upper() in intersect], 
                       DNA, DNA_features, ZDNA, test_intervals,
                          DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False))
    
    dump((train_dataset, test_dataset), f'ds_w_seq_hg_fold{MODEL_NUMBER}.pkl')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected 

In [6]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        features, labels, sequences, input_ids, intervals = batch            
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        #print(model.device, input_ids.device, labels.device)
        outputs = model(input_ids = input_ids, labels = labels)        
        #print(outputs)
        loss, tr_logits = outputs['loss'], outputs['logits']
        #print(model(input_ids=ids, attention_mask=mask, labels=labels))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 1000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 1000 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=0.1
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [7]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_scores = [], [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            features, labels, sequences, input_ids, intervals = batch            
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model(input_ids = input_ids, labels = labels)
            loss, eval_logits = outputs['loss'], outputs['logits']            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            flattened_scores = active_logits[:,1] - active_logits[:,0]
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_scores.extend(flattened_scores)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id.item() for id in eval_labels]
    predictions = [id.item() for id in eval_preds]
    scores = [id.item() for id in eval_scores]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, scores

In [8]:
device = 3
lr = 1e-5
EPOCHS = 4

dir_to_pretrained_model = "6-new-12w-0/"
config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')
tokenizer = DNATokenizer.from_pretrained('6-new-12w-0/')

for MODEL_NUMBER in range(5):
    train_dataset, test_dataset = load(f'ds_w_seq_hg_fold{MODEL_NUMBER}.pkl')
    training_loader = DataLoader(train_dataset, batch_size=24, num_workers = 2)
    testing_loader = DataLoader(test_dataset, batch_size=16, num_workers = 2)


    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config)
    model.to(device)
    
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(training_loader), epochs=EPOCHS)

    for epoch in range(EPOCHS):
        print(f"Training fold {MODEL_NUMBER} epoch: {epoch + 1}")
        train(epoch)
        labels, predictions, scores = valid(model, testing_loader)
        print(f'Fold {MODEL_NUMBER} validation ROC-AUC: ', roc_auc_score(labels, scores))

    print(sklearn.metrics.classification_report(labels, predictions))
    model.save_pretrained(f'dnabert_hg_fold_{MODEL_NUMBER}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.
Some weights of the model checkpoint at 6-new-12w-0/ were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint

Training fold 0 epoch: 1


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss per 1000 training steps: 0.6781904101371765
Training loss epoch: 0.26648056231435946
Training accuracy epoch: 0.8756896096112052


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Validation loss per 100 evaluation steps: 0.04987993836402893
Validation loss per 100 evaluation steps: 0.04196281087156016
Validation Loss: 0.042986117465371065
Validation Accuracy: 0.9849110798663404
Fold 4 validation ROC-AUC:  0.9511417602132829
Training fold 4 epoch: 4


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training loss per 1000 training steps: 0.07433884590864182
Training loss epoch: 0.04078822878538895
Training accuracy epoch: 0.9858110561933107


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Validation loss per 100 evaluation steps: 0.04861769452691078
Validation loss per 100 evaluation steps: 0.04192888862245928
Validation Loss: 0.04254314429912133
Validation Accuracy: 0.9850890377917921
Fold 4 validation ROC-AUC:  0.9532913205902089
              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1324295
           1       0.70      0.49      0.57     27897

    accuracy                           0.99   1352192
   macro avg       0.84      0.74      0.78   1352192
weighted avg       0.98      0.99      0.98   1352192



In [17]:
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]
all_features = sorted([i[:-4] for i in os.listdir('../data/hg19_features/sparse/') if i.endswith('.pkl')])
groups = ['DNase-seq', 'Histone', 'RNA polymerase', 'TFs and others']
feature_names = [i for i in all_features if (i.split('_')[0] in groups)]
ZDNA = load(f'../data/hg19_zdna/sparse/ZDNA_2016.pkl')
black_list = load(f'../data/hg19_zdna/sparse/blacklist_hg19.pkl')

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])

DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

DNA_features = {feture: load(f'../data/hg19_features/sparse/{feture}.pkl')
                for feture in tqdm(feature_names)}

for feature in tqdm(DNA_features):
    if set(DNA_features[feature].keys()) != set(chroms):
        for chrom in chroms:
            if chrom not in DNA_features[feature]:
                DNA_features[feature][chrom] = SparseVector(len(DNA[chrom]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1054/1054 [00:00<00:00, 77419.29it/s]


In [18]:
device = 3
torch.cuda.empty_cache()
models = []

config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')

for MODEL_NUMBER in range(5):
    dir_to_pretrained_model = f'dnabert_hg_fold_{MODEL_NUMBER}'
    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config).to(device)
    model.to(device)
    model.eval()
    models.append(model)

In [21]:
tokenizer = DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.


In [22]:
width = 512
k_mer_pad = 5

def final_prediction(chrom):
    
    intervals = []
    ends = []
    
    prediction = [np.zeros(ZDNA[chrom].shape, dtype=np.float32) for i in range(5)]
    
    
    for st in range(0, ZDNA[chrom].shape - width, width):
        interval = [st, min(st + width + k_mer_pad, ZDNA[chrom].shape)]
        intervals.append([chrom, interval[0], interval[1]])
        
    ends.append([chrom, ZDNA[chrom].shape // width * width, ZDNA[chrom].shape])


    full_dataset = Dataset(chroms, [i for i in feature_names if i.upper() in intersect], 
                           DNA, DNA_features,  ZDNA, intervals, tokenizer)

    ends_dataset = Dataset(chroms, [i for i in feature_names if i.upper() in intersect], 
                           DNA, DNA_features, ZDNA, ends, tokenizer)

    params = {'batch_size':32, 'num_workers':5, 'shuffle':False}
    load_predict = data.DataLoader(full_dataset, **params)

    params = {'batch_size':1, 'num_workers':1, 'shuffle':False}
    load_ends = data.DataLoader(ends_dataset, **params)
    
    with torch.no_grad():
        
        for features, labels, sequences, input_ids, intervals in tqdm(load_predict):
            input_ids = input_ids.to(device)
            outputs = [torch.softmax(model(input_ids = input_ids)['logits'],axis = -1).cpu().numpy()[:,:,1] for model in models]
            
            for ind, interval in enumerate(zip(intervals[0], 
                                         intervals[1].cpu().numpy(),
                                         intervals[2].cpu().numpy())):
                #print(ind, interval)
                #print(len(prediction), len(outputs), len(models))
                for i in range(len(models)):
                    prediction[i][interval[1]:interval[2]-k_mer_pad] = outputs[i][ind, :]
                    
    for model_i in range(len(models)):
        dump(prediction[model_i], f'../models/predictions/new_mod_hg_{model_i}_{chrom}', 3)
                         

In [ ]:
for chrom in chroms[19:]:
    print(f"BEGIN CHROM {chrom}")
    final_prediction(chrom)

BEGIN CHROM chr20


  0%|                                                                                                                         | 0/3847 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `trun

In [ ]:
chroms = [f'chr{i}' for i in list(range(8, 23)) + ['X', 'Y']]

In [ ]:
ZDNA = load(f'../data/hg19_zdna/sparse/ZDNA_2016.pkl')
black_list = load(f'../data/hg19_zdna/sparse/blacklist_hg19.pkl')

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

In [ ]:
equalized, divisions = load('hg_divisions.pkl')

In [ ]:
com_len = sum([len(DNA[chrom]) for chrom in chroms])
sums = []

for chrom in tqdm(chroms):
    loc_sum = []
    for model_num in range(5):
        vec = load(f"../models/predictions/new_mod_hg_{model_num}_{chrom}")
        loc_sum.append(vec.sum())
    sums.append(loc_sum)

multipliers = np.array(sums).sum(axis=0) / com_len

In [ ]:
for chrom in tqdm(chroms):
    vecs = np.array([load(f"../models/predictions/new_mod_hg_{i}_{chrom}") 
                     for i in range(5)])
    res_vec = (vecs / multipliers[:, None]) * multipliers.mean()
    mean_vec = res_vec.mean(axis=0)
    
    test_ints = []
    for MODEL_NUMBER in range(5):
        train_inds, test_inds = divisions[MODEL_NUMBER]
        train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]
        test_ints.extend([(MODEL_NUMBER, inter) for inter in test_intervals if inter[0] == chrom])
    
    for model_num, inters in test_ints:
        mean_vec[inters[1]: inters[2]] = res_vec[model_num, inters[1]: inters[2]]
    dump(mean_vec, f"../models/predictions/new_mod_hg_res_{chrom}", 3)

In [ ]:
for chrom in tqdm(chroms):
    vec = load(f"../models/predictions/new_mod_hg_res_{chrom}")
    iids = np.where(black_list[chrom].data == 1)[0]
    for i, j in zip(black_list[chrom].indices[iids], black_list[chrom].indices[iids + 1]):
        vec[i:j] = 0
    dump(vec, f"../models/predictions/new_mod_hg_res_{chrom}", 3)